In [1]:
import requests
from datetime import date, datetime
import xmltodict
import json

In [2]:
today = datetime.today()
today_string = datetime.strftime(today, '%d-%m-%Y')

URL = 'https://danhmuchanhchinh.gso.gov.vn/DMDVHC.asmx?wsdl'

In [3]:
WARD_HEADERS = {
    'Content-Type': 'text/xml',
    'SOAPAction': "http://tempuri.org/DanhMucPhuongXa"
} 

WARD_BODY = (f''
'<?xml version="1.0" encoding="utf-8"?>'
'<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
'  <soap:Body>'
'    <DanhMucPhuongXa xmlns="http://tempuri.org/">'
f'      <DenNgay>{today_string}</DenNgay>'
#'      <Tinh>string</Tinh>'
#'      <TenTinh>string</TenTinh>'
#'      <QuanHuyen>string</QuanHuyen>'
#'      <TenQuanHuyen>string</TenQuanHuyen>'
'    </DanhMucPhuongXa>'
'  </soap:Body>'
'</soap:Envelope>'
)

In [4]:
def return_value_with_key(root : dict, key: str):
    result = None
    
    if type(root) is not dict:
        return result
    
    for k, v in root.items():
        if k == key:
            return v
        else:
            result = return_value_with_key(v, key=key)
            if result is not None:
                return result
    return result


def get_danhmuchanhchinh_response(
    body : str,
    headers : str,
    url : str = URL
) -> requests.Response:
    """
    Retrieve utf-8 decoded content of response from DanhMucHanhChinh API
    based on passed-in body and header post.
    """

    try:
        response = requests.post(url,data=body,headers=headers)
    except Exception as e:
        print(f"error {e} in get_province()")
    else:
        return response.content.decode("utf-8")

def get_everything():
    """
    Get the most complete responses from DanhMucHanhChinh API for 
    all administrative subdivisions. parse response and save content
    into json format.
    """
    # get province
    # province_content = get_danhmuchanhchinh_response(PROVINCE_BODY, PROVINCE_HEADERS)
    # provinces = parse_province_content(province_content)
    # with open('provinces.json', 'w', encoding='utf8') as provinces_json_name:
    #     json.dump(provinces, provinces_json_name, ensure_ascii=False, indent=4)

    
    # # get district
    # district_content = get_danhmuchanhchinh_response(DISTRICT_BODY, DISTRICT_HEADERS)
    # districts = parse_district_content(province_content)
    
    # get wards
    ward_content = get_danhmuchanhchinh_response(WARD_BODY, WARD_HEADERS)
    wards = parse_ward_content(province_content)
    return 0

In [5]:
ward_content = get_danhmuchanhchinh_response(WARD_BODY, WARD_HEADERS)

NameError: name 'district_content' is not defined

In [6]:
ward_content

'<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><DanhMucPhuongXaResponse xmlns="http://tempuri.org/"><DanhMucPhuongXaResult><xs:schema id="NewDataSet" xmlns="" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:msdata="urn:schemas-microsoft-com:xml-msdata"><xs:element name="NewDataSet" msdata:IsDataSet="true" msdata:MainDataTable="TABLE" msdata:UseCurrentLocale="true"><xs:complexType><xs:choice minOccurs="0" maxOccurs="unbounded"><xs:element name="TABLE"><xs:complexType><xs:sequence><xs:element name="MaTinh" type="xs:string" minOccurs="0" /><xs:element name="TenTinh" type="xs:string" minOccurs="0" /><xs:element name="MaQuanHuyen" type="xs:string" minOccurs="0" /><xs:element name="TenQuanHuyen" type="xs:string" minOccurs="0" /><xs:element name="MaPhuongXa" type="xs:string" minOccurs="0" /><xs:element name="TenPhuongXa" 

In [14]:
def parse_ward_content(ward_content):
    ward_content_dict = xmltodict.parse(ward_content)
    ward_dict = return_value_with_key(root=ward_content_dict, key='TABLE')

    wards = []
    for item in ward_dict:
        ward = {}

        keys = ("MaTinh", "TenTinh", "MaQuanHuyen", "TenQuanHuyen", "MaPhuongXa", "TenPhuongXa", "LoaiHinh", "@msdata:rowOrder", "@diffgr:id")
        for key in item:
            if key not in keys:
                print(f"found weird key: {key}")
                print(f"    with values: {item[key]}")

        ward["province_service_key"] = item["MaTinh"]
        ward["province_name"] = item["TenTinh"]
        ward["district_service_key"] = item["MaQuanHuyen"]
        ward["district_name"] = item["TenQuanHuyen"]
        ward["ward_service_key"] = item["MaPhuongXa"]
        ward["ward_name"] = item["TenPhuongXa"]
        ward["type"] = item["LoaiHinh"]
        ward["ward_service_order"] = item["@msdata:rowOrder"]
    
        wards.append(ward)
        
    return wards

In [15]:
wards = parse_ward_content(ward_content)
wards
ai là luật sư đại diện em ạ?

[{'province_service_key': '01',
  'province_name': 'Thành phố Hà Nội',
  'district_service_key': '001',
  'district_name': 'Quận Ba Đình',
  'ward_service_key': '00001',
  'ward_name': 'Phường Phúc Xá',
  'type': 'Phường',
  'province_service_order': '0'},
 {'province_service_key': '01',
  'province_name': 'Thành phố Hà Nội',
  'district_service_key': '001',
  'district_name': 'Quận Ba Đình',
  'ward_service_key': '00004',
  'ward_name': 'Phường Trúc Bạch',
  'type': 'Phường',
  'province_service_order': '1'},
 {'province_service_key': '01',
  'province_name': 'Thành phố Hà Nội',
  'district_service_key': '001',
  'district_name': 'Quận Ba Đình',
  'ward_service_key': '00006',
  'ward_name': 'Phường Vĩnh Phúc',
  'type': 'Phường',
  'province_service_order': '2'},
 {'province_service_key': '01',
  'province_name': 'Thành phố Hà Nội',
  'district_service_key': '001',
  'district_name': 'Quận Ba Đình',
  'ward_service_key': '00007',
  'ward_name': 'Phường Cống Vị',
  'type': 'Phường',
  